In [1]:
import pyFAI
import pyFAI.calibrant
import pyFAI.detectors
import os, glob, re
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox
import ipywidgets as widgets
from tifffile import imread, imshow, imsave
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import gridspec
from matplotlib.widgets import Slider, Button
import yaml, tifffile
%matplotlib widget

def sum_everything(my_im1, my_im2, my_im3, use_mask_1, use_mask_2, use_mask_3, osetx = 27, osety = 27):
    """ Assuming im2 offset by -osetx, -osety, and im3 offset by +osetx, +osety """
    
    my_imsum = np.ones((my_im1.shape[0]+int(2*osetx), my_im2.shape[1]+int(2*osety),3))*np.nan

    my_imsum[osetx:-osetx,osety:-osety,0] = my_im1
    my_imsum[osetx:-osetx,osety:-osety,0][use_mask_1==1] = np.nan

    my_imsum[:-int(2*osetx),:-int(2*osety):,1] = my_im2
    my_imsum[:-int(2*osetx),:-int(2*osety):,1][use_mask_2==1] = np.nan

    my_imsum[int(2*osetx):,int(2*osety):,2] = my_im3
    my_imsum[int(2*osetx):,int(2*osety):,2][use_mask_3==1] = np.nan
    return np.nanmean(my_imsum,axis=2)

def image_sum():
    data_dir = tiff_base_path + "dark_sub/"
    meta_dir = tiff_base_path + "meta/"
    Data = []
    
    os.chdir(data_dir)
    Tiff, Meta = [], []
    for file in os.listdir(data_dir):
        if file.endswith(".tiff"):
            Tiff.append(file)
    #A.sort(key=lambda x: os.path.getmtime(x))
    for file in os.listdir(meta_dir):
        if file.endswith(".yaml"):
            Meta.append(file)
    Tiff.sort()
    Meta.sort()
    F_number = len(Tiff)
    print(F_number)
    print(len(Meta))

    File_Name_Prefix = folder_name = os.path.basename(tiff_base_path.rstrip('/'))
    sum_dir = os.path.join(data_dir, 'sum')
    os.makedirs(sum_dir, exist_ok=True)  # Create 'sum' directory if it doesn't exist
    
    
    for i in range(1, F_number, 3):  # Iterate with a step of 3
        if (i+2) >= F_number:
            break
        im_spotA = imread(Tiff[i+0])
        im_spotB = imread(Tiff[i+1])
        im_spotC = imread(Tiff[i+2])
    
        with open(os.path.join(meta_dir, Meta[i]), 'r') as f:
            data = yaml.unsafe_load(f)
            T = data.get('more_info')['cryostat_A']
    
        full_imsum = sum_everything(im_spotC, im_spotB, im_spotA, use_mask_3, use_mask_2, use_mask_1, osety=27, osetx=27)
        
        # Save output file to the new 'sum' directory
        tifffile.imsave(f"{sum_dir}/{File_Name_Prefix}_Temperature_{T}_K_sum.tiff", full_imsum)
    print (i)
    print (list(Tiff))


def pct_integration():
    i2d, q2d, chi2d = ai.integrate2d(img, binning, unit=UNIT,npt_azim=3600, polarization_factor=polarization, mask=mask0) # perform azimuthalintegration on one image to retain 2D information
    intrinsic_mask_unrolled,_,_ = ai.integrate2d(mask0, binning, unit=UNIT,npt_azim=3600, polarization_factor=polarization, mask=mask0)  #trasnform mask0 (base mask) to the same coordinate space and cast it as type bool
    #intrinsic_mask_unrolled = intrinsic_mask_unrolled.astype(bool) 
    outlier_mask_2d = np.zeros_like(i2d)     # Create an array to hold outlier mask

    i2d, q2d, chi2d = ai.integrate2d(img, binning, unit=UNIT, npt_azim=3600, polarization_factor=polarization, mask=mask0)
    mask1 = np.array(i2d<1)*1
    
    for ii, dd in enumerate(i2d.T):
        low_limit, high_limit = np.percentile(dd, (ll,ul))
        outlier_mask_2d[:,ii] = np.any([dd<low_limit, dd>high_limit, intrinsic_mask_unrolled[:,ii]], axis=0)
    mask=outlier_mask_2d + mask1    
    outlier_mask_2d_masked =  ma.masked_array(i2d, mask=outlier_mask_2d + mask1)     
    data = np.column_stack((q2d, ma.mean(outlier_mask_2d_masked,axis=0)))
    #plt.plot(q2d, ma.mean(outlier_mask_2d_masked,axis=0))
    np.savetxt(directory + "/dark_sub/sum/" + str(a) + "_L" + str(ll)+ "_U" + str(ul) + "_percentile_masked_"+ str(UNIT) + ".dat", data) # Uncomment when the data is ready to be saved


ModuleNotFoundError: No module named 'pyFAI'

In [ ]:
# Below stiches every three pilatus images in folders that end with 'PDF' in the tiff_base
tiff_base_path = ('/nsls2/data/pdf/pdfhack/legacy/processed/xpdacq_data/user_data_Petkov_315831_79f731e2_2025-04-09-1101//tiff_base/')    
use_mask_1= np.load("/nsls2/data/pdf/pdfhack/legacy/processed/xpdacq_data/user_data_Frandsen_315944_8496f7ec_2025-04-12-1911//config_base/masks/Mask_pos1_ext_BS.npy") # This is mask we are applying befor mergin images
use_mask_2 = np.load("/nsls2/data/pdf/pdfhack/legacy/processed/xpdacq_data/user_data_Frandsen_315944_8496f7ec_2025-04-12-1911//config_base/masks/Mask_pos2_ext_BS.npy") # This is mask we are applying befor mergin images
use_mask_3 = np.load("/nsls2/data/pdf/pdfhack/legacy/processed/xpdacq_data/user_data_Frandsen_315944_8496f7ec_2025-04-12-1911//config_base/masks/Mask_pos3_ext_BS.npy") # This is mask we are applying befor mergin images

# Get all folders ending with ending with '_PDF'. Alternativelly if you want to select a perticalur folder Type the folder name instead of "*_PDF"
pdf_folders = glob.glob(tiff_base_path + "*_PDF/")
for item in pdf_folders:
    print(item)

for directory in pdf_folders:
    print(directory)
    tiff_base_path = directory
    image_sum()

In [ ]:
ai = pyFAI.load("/nsls2/data/pdf/pdfhack/legacy/processed//xpdacq_data/user_data_Frandsen_315944_8496f7ec_2025-04-12-1911/config_base/merged_PDF/merged.poni" ) 
mask0 = np.load("/nsls2/data/pdf/pdfhack/legacy/processed//xpdacq_data/user_data_Frandsen_315944_8496f7ec_2025-04-12-1911/config_base/masks/stitched_2.npy")
binning = 5000
polarization = 0.99
UNIT = "q_A^-1"
ll = 1 # lower percentile for the q dependent mask
ul = 99  # Upper percentile for the q dependent mask


In [ ]:
# Integrate applying a percentile + base mask and save 2 column data. Percentile masking remove outliers based on the intensity at each Q independently 

pdf_folders = glob.glob(tiff_base_path + "*_PDF/") # PDF folder selection

for directory in pdf_folders:
    print(directory)
    for file in os.listdir(directory + "/dark_sub/sum/"):
        #print(directory)
        if file.endswith(".tiff"):
            img = imread(directory + "/dark_sub/sum/" + file)
            print(file)
            a,b = os.path.splitext(file)
            pct_integration()


In [ ]:
# Integrate only applying a base_mask and save 2 column data) 
pdf_folders = glob.glob(tiff_base_path + "*_PDF/")
#for item in pdf_folders:
#    print(item)

for directory in pdf_folders:
    print(directory)
    for file in os.listdir(directory + "/dark_sub/sum/"):
        #print(directory)
        if file.endswith(".tiff"):
            img = imread(directory + "/dark_sub/sum/" + file)
            a,b = os.path.splitext(file)
            x,y = ai.integrate1d(img, binning, mask=mask0, unit=UNIT, polarization_factor=polarization) 
            data = np.column_stack((x, y))
            np.savetxt(directory + "/dark_sub/sum/" + str(a) + ".dat", data)